Punto 2
 - Jacobo Aristizábal, Isabella García & María Giraldo

In [108]:
#Importar librerías
import random
import numpy as np
import matplotlib.pyplot as plt

from deap import base, creator, tools, algorithms

import warnings
warnings.filterwarnings('ignore')

In [109]:
#Definir las ciudades
ciudades = ["Medellín", "Bogotá", "Cartagena", "Cali", "Bucaramanga"]

#Distancias entre las ciudades en km 
distancias = {
    "Medellín": {"Bogotá": 416, "Cartagena": 640, "Cali": 409, "Bucaramanga": 380},
    "Bogotá": {"Medellín": 416, "Cartagena": 1041, "Cali": 508, "Bucaramanga": 426},
    "Cartagena": {"Medellín": 640, "Bogotá": 1041, "Cali": 1046, "Bucaramanga": 622},
    "Cali": {"Medellín": 409, "Bogotá": 508, "Cartagena": 1046, "Bucaramanga": 766},
    "Bucaramanga": {"Medellín": 380, "Bogotá": 426, "Cartagena": 622, "Cali": 766}
}


In [110]:
#Declarar el contenedor de operadores genéticos
toolbox = base.Toolbox()

In [111]:
#Crear una clase para la función de adaptabilidad y para el individuo
creator.create("FitnessMin", base.Fitness, weights = (-1.0,))
creator.create("Individual", list, fitness = creator.FitnessMin)

In [112]:
#Distancia total recorrida
def distanciaTotal(individuo):
    distancia = 0
    for i in range (numeroCiudades - 1):
        ciudadActual = individuo[i]
        ciudadSiguiente = individuo[i + 1]
        distancia += distancias[ciudadActual][ciudadSiguiente]
    distancia += distancias[individuo[-1]][individuo[0]]
    return distancia,

In [113]:
#Cromosomas booleanos
toolbox.register("attribute", random.sample, ciudades, len(ciudades))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attribute)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#Operadores genéticos
toolbox.register('select', tools.selTournament, tournsize = 3)
toolbox.register('mate', tools.cxOnePoint)
toolbox.register('mutate', tools.mutFlipBit, indpb = 0.2)

#Evaluación de adaptabilidad
toolbox.register("evaluate", distanciaTotal)

In [114]:
#Parámetros del algoritmo
numeroCiudades = len(ciudades)
tamañoPoblacion = 50
numeroGeneraciones = 100
probabilidadCruce = 0.5
probabilidadMutacion = 0.01

In [115]:
#Iniciar la población 
poblacion = toolbox.population(n = tamañoPoblacion)

In [116]:
#Algoritmo genético y evaluación de la función de aptidud
for generacion in range(numeroGeneraciones):
    aptitudes = [toolbox.evaluate(individuo) for individuo in poblacion]
    for individuo, aptitud in zip(poblacion, aptitudes):
        individuo.fitness.values = aptitud
    desc = toolbox.select(poblacion, len(poblacion))
    desc = [toolbox.clone(individuo) for individuo in desc]
    
    for hijo1, hijo2 in zip(desc[::2], desc[1::2]):
        if random.random() < probabilidadCruce:
            toolbox.mate(hijo1, hijo2)

    for individuo in desc:
        if random.random() < probabilidadMutacion:
            toolbox.mutate(individuo)

In [117]:
#Nueva generación
poblacion[:] = desc

In [118]:
#Mejor ruta a tomar
mejorInd = tools.selBest(poblacion, k=1)[0]
menorDist = distanciaTotal(mejorInd)[0]
print("Menor distancia: {}".format(menorDist))
print("Ruta óptima:")
print(mejorInd)

Menor distancia: 2093
Ruta óptima:
['Cali', 'Medellín', 'Bucaramanga', 'Medellín', 'Bogotá']
